In [326]:
import pandas as pd
from googleapiclient.discovery import build
from google.oauth2 import service_account

In [327]:
import numpy as np
import datetime
import pandasql

## I. Download Tables from Google SpreadSheet

In [328]:
SERVICE_ACCOUNT_FILE = 'key.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
creds = None
creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1Ycg7zTxds9DZnDvTrFcyNNKuTUxg6Yy6WF0a8Wc02WQ'

In [329]:
service = build('sheets', 'v4', credentials=creds)

# Call the Sheets API
sheet = service.spreadsheets()

### Table Transactions

In [330]:
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range="transactions!A1:D29001").execute()
values = result.get('values', [])

In [331]:
transactions = pd.DataFrame(values[1:], columns = values[0]) 

In [332]:
transactions.head()

,transaction_id,created_at,m_real_amount,l_client_id
0,2c9f1527-8e7f-4fb1-8000-c747a2ab46c9,2020-09-30 07:15:14,31,8a805e60-6fd6-11e7-80fc-c412f533dba1
1,2c9f1527-8e7f-4fb1-8000-c747a2ab46c9,2020-09-30 07:15:14,87,8a805e60-6fd6-11e7-80fc-c412f533dba1
2,ab8cbcf7-3271-49a0-8001-6cf9816f63b8,2020-09-12 13:47:04,49231,dc0d0e52-629a-11ea-abf3-c412f533dba1
3,ab8cbcf7-3271-49a0-8001-6cf9816f63b8,2020-09-12 13:47:04,17305,dc0d0e52-629a-11ea-abf3-c412f533dba1
4,52f8ebcc-d82d-4be1-8004-72cfbe0dff24,2020-09-25 06:00:07,9022,e663b6a0-4a1a-11e8-a2c3-c412f533dba1


#### Check for the missing values

In [334]:
#transactions.to_csv('transactions.csv', header=True, index=False)
transactions.isna().sum()

transaction_id    0
created_at        0
m_real_amount     0
l_client_id       0
dtype: int64

#### Check types

In [335]:
transactions.dtypes

transaction_id    object
created_at        object
m_real_amount     object
l_client_id       object
dtype: object

#### Convert date to datetime type and amount to numeric, because we will do mathematical operations with them

In [336]:
transactions['created_at'] = pd.to_datetime(transactions['created_at'])

In [337]:
transactions['m_real_amount'] = pd.to_numeric(transactions['m_real_amount'])

### Table Clients

In [338]:
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range="clients!A1:C75767").execute()
values = result.get('values', [])

In [339]:
clients = pd.DataFrame(values[1:], columns = values[0]) 

In [340]:
clients.head()

,client_id,created_at,l_manager_id
0,0001da7d-fcda-11ea-ac18-c412f533dba1,2020-09-22 17:46:23,9a87c608-52dc-11ea-abeb-c412f533dba1
1,000a8743-9ae1-11e7-8114-c412f533dba1,2017-09-16 00:00:00,ad52c7a8-a752-11e7-8115-c412f533dba1
2,000ae57e-2d48-11ea-abeb-c412f533dba1,2020-01-02 14:08:56,43756fa4-57a0-11e9-ab9a-c412f533dba1
3,00133cde-481c-11ea-abeb-c412f533dba1,2020-02-05 17:25:13,9a87c608-52dc-11ea-abeb-c412f533dba1
4,00148d3f-07e6-11e8-812a-c412f533dba1,2018-02-02 00:00:00,abed5496-ca88-11e9-abb8-c412f533dba1


#### Check for the missing values

In [341]:
#clients.to_csv('clients.csv', header=True, index=False)
clients.isna().sum()

client_id        0
created_at      42
l_manager_id    42
dtype: int64

#### Check types

In [342]:
clients.dtypes

client_id       object
created_at      object
l_manager_id    object
dtype: object

We see, that there is date 0001-01-01 00:00:00 is used as N/A. Therefore, we can fill N/A and this value by the following date: 01/01/1900

In [343]:
clients['created_at'] = pd.to_datetime(clients['created_at'], errors = 'coerce')
clients['created_at'] = clients['created_at'].fillna(datetime.datetime(1900, 1, 1))

To fill in the missing values we have to look at the other tables with manager_id. In table LEADS, we see the use of 00000000-0000-0000-0000-000000000000 as the undefined manager_id, so we can use it as well

In [344]:
clients['l_manager_id'] = clients['l_manager_id'].fillna('00000000-0000-0000-0000-000000000000')

## Table Managers

In [345]:
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range="managers!A1:C14").execute()
values = result.get('values', [])

In [346]:
managers = pd.DataFrame(values[1:], columns = values[0]) 

This table is small, so we can look at the whole table

In [347]:
#managers.to_csv('managers.csv', header=True, index=False)
managers

,manager_id,d_manager,d_club
0,1e9c5562-0cfc-11ea-abe1-c412f533dba1,manager #8,club #2
1,43756fa4-57a0-11e9-ab9a-c412f533dba1,manager #11,club #4
2,543888c4-fbb3-11e9-abdc-c412f533dba1,manager #12,club #3
3,5dd88f32-6501-11ea-abf3-c412f533dba1,manager #10,club #2
4,9a87c608-52dc-11ea-abeb-c412f533dba1,manager #9,club #3
5,abed5496-ca88-11e9-abb8-c412f533dba1,manager #5,club #4
6,ad52c7a8-a752-11e7-8115-c412f533dba1,manager #7,club #1
7,b55a233c-5df6-11ea-abf2-c412f533dba1,manager #2,club #4
8,b832aac3-fcda-11ea-ac18-c412f533dba1,manager #13,club #3
9,d12fe765-b444-11e9-abb3-c412f533dba1,manager #1,club #1


### Table Leads

In [348]:
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range="leads!A1:F3338").execute()
values = result.get('values', [])

In [349]:
leads = pd.DataFrame(values[1:], columns = values[0]) 

In [350]:
leads.head()

,lead_id,created_at,d_utm_medium,d_utm_source,l_manager_id,l_client_id
0,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1
1,0078b05d-0f1f-11eb-ac1a-c412f533dba1,2020-10-15 23:45:40,cpc,instagram,d12fe765-b444-11e9-abb3-c412f533dba1,606c5b29-0f88-11eb-ac1a-c412f533dba1
2,007d1401-2af3-11eb-ac1f-c412f533dba1,2020-11-20 09:41:14,cpc,instagram,43756fa4-57a0-11e9-ab9a-c412f533dba1,1f389c6a-d5ab-11e9-abbe-c412f533dba1
3,0084e614-2fcc-11eb-ac1f-c412f533dba1,2020-11-26 13:44:40,,,00000000-0000-0000-0000-000000000000,00000000-0000-0000-0000-000000000000
4,009b1616-145d-11eb-ac1d-c412f533dba1,2020-10-22 15:52:04,cpc,vk,5dd88f32-6501-11ea-abf3-c412f533dba1,ac2c0012-6fd5-11e7-80fc-c412f533dba1


#### Check for the missing values

In [351]:
#leads.to_csv('leads.csv', header=True, index=False)
leads.isna().sum()

lead_id         0
created_at      0
d_utm_medium    0
d_utm_source    0
l_manager_id    0
l_client_id     0
dtype: int64

#### Check types

In [352]:
leads.dtypes

lead_id         object
created_at      object
d_utm_medium    object
d_utm_source    object
l_manager_id    object
l_client_id     object
dtype: object

#### Convert date to datetime type

In [353]:
leads['created_at'] = pd.to_datetime(leads['created_at'])

## II. Preparation of the Report

### Set the time period

In [354]:
x = datetime.datetime(2020, 10, 17)
y = datetime.datetime(2020, 11, 17)

In [355]:
d = {'x': [x], 'y': [y]}
period = pd.DataFrame(data=d)
period

,x,y
0,2020-10-17,2020-11-17


#### Check the source column

In [356]:
leads['d_utm_source'].unique()

array(['viber', 'instagram', '', 'vk', 'sms', 'google', 'utm_source',
       'insta', 'vkontakte', 'yandex', 'ycard',
       'ycard#!/tproduct/225696739-1498486363994', 'ig'], dtype=object)

In [357]:
condition = leads['d_utm_source'].str.contains('ycard')
leads['d_utm_source'] = np.where(condition, 'ycard', leads['d_utm_source'])
condition = leads['d_utm_source'].str.contains('insta')
leads['d_utm_source'] = np.where(condition, 'instagram', leads['d_utm_source'])
condition = leads['d_utm_source'].str.contains('vk')
leads['d_utm_source'] = np.where(condition, 'vk', leads['d_utm_source'])

In [358]:
leads['d_utm_source'].unique()

array(['viber', 'instagram', '', 'vk', 'sms', 'google', 'utm_source',
       'yandex', 'ycard', 'ig'], dtype=object)

In [359]:
leads['d_utm_source'].replace('', 'unknown', inplace=True)
leads['d_utm_source'].replace('utm_source', 'unknown', inplace=True)

In [360]:
leads.head()

,lead_id,created_at,d_utm_medium,d_utm_source,l_manager_id,l_client_id
0,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1
1,0078b05d-0f1f-11eb-ac1a-c412f533dba1,2020-10-15 23:45:40,cpc,instagram,d12fe765-b444-11e9-abb3-c412f533dba1,606c5b29-0f88-11eb-ac1a-c412f533dba1
2,007d1401-2af3-11eb-ac1f-c412f533dba1,2020-11-20 09:41:14,cpc,instagram,43756fa4-57a0-11e9-ab9a-c412f533dba1,1f389c6a-d5ab-11e9-abbe-c412f533dba1
3,0084e614-2fcc-11eb-ac1f-c412f533dba1,2020-11-26 13:44:40,,unknown,00000000-0000-0000-0000-000000000000,00000000-0000-0000-0000-000000000000
4,009b1616-145d-11eb-ac1d-c412f533dba1,2020-10-22 15:52:04,cpc,vk,5dd88f32-6501-11ea-abf3-c412f533dba1,ac2c0012-6fd5-11e7-80fc-c412f533dba1


Check that all the clients with id in the table leads are in the table clients

In [361]:
sub_data1 = pandasql.sqldf("SELECT DISTINCT leads.l_client_id FROM leads LEFT JOIN clients ON leads.l_client_id = clients.client_id;", globals())
sub_data2 = pandasql.sqldf("SELECT DISTINCT leads.l_client_id FROM leads INNER JOIN clients ON leads.l_client_id = clients.client_id;", globals()) 
pd.concat([sub_data1,sub_data2]).drop_duplicates(keep=False)

,l_client_id
3,00000000-0000-0000-0000-000000000000


Check that all the clients with transactions are in the table clients

In [362]:
sub_data1 = pandasql.sqldf("SELECT DISTINCT transactions.l_client_id FROM transactions LEFT JOIN clients ON transactions.l_client_id = clients.client_id;", globals())
sub_data2 = pandasql.sqldf("SELECT DISTINCT transactions.l_client_id FROM transactions INNER JOIN clients ON transactions.l_client_id = clients.client_id;", globals()) 
pd.concat([sub_data1,sub_data2]).drop_duplicates(keep=False)

,l_client_id


Check that all clients without manager in leads and in transactions have managers

In [363]:
client_no_manager = pandasql.sqldf("SELECT l_client_id FROM leads WHERE l_manager_id = '00000000-0000-0000-0000-000000000000' AND l_client_id <> '00000000-0000-0000-0000-000000000000';", globals())

In [364]:
clients.loc[clients['client_id'].isin(client_no_manager['l_client_id'])]

,client_id,created_at,l_manager_id
2381,086ee5d4-620d-11ea-abf3-c412f533dba1,2020-03-09 17:50:43,abed5496-ca88-11e9-abb8-c412f533dba1
8678,1d9a669a-d5a5-11e9-abbe-c412f533dba1,1900-01-01 00:00:00,db06b8bc-bfb2-11e7-8116-c412f533dba1
8725,1da0523f-d5b5-11e9-abbe-c412f533dba1,1900-01-01 00:00:00,9a87c608-52dc-11ea-abeb-c412f533dba1
11132,251d41aa-c96b-11e9-abb7-c412f533dba1,2019-08-28 00:00:00,bf340611-034b-11eb-ac18-c412f533dba1
11327,25bda8e3-d5ac-11e9-abbe-c412f533dba1,1900-01-01 00:00:00,9a87c608-52dc-11ea-abeb-c412f533dba1
13139,2b0c485e-d5a6-11e9-abbe-c412f533dba1,1900-01-01 00:00:00,1bb0e7ca-5892-11ea-abeb-c412f533dba1
13239,2b361f83-0fbd-11eb-ac1a-c412f533dba1,2020-10-16 18:37:52,43756fa4-57a0-11e9-ab9a-c412f533dba1
15036,31b2d371-0a06-11eb-ac1a-c412f533dba1,2020-10-09 12:05:36,abed5496-ca88-11e9-abb8-c412f533dba1
17464,394eadaf-19dc-11eb-ac1d-c412f533dba1,2020-10-29 15:45:21,d6a49e22-f74f-11ea-ac18-c412f533dba1
18866,3d80f361-302e-11e8-8131-c412f533dba1,2018-03-25 00:00:00,b55a233c-5df6-11ea-abf2-c412f533dba1


In [365]:
client_and_manager = pandasql.sqldf("SELECT DISTINCT l_client_id, l_manager_id FROM leads;", globals())
client_and_manager

,l_client_id,l_manager_id
0,7264cf21-a31d-11e9-abb0-c412f533dba1,1e9c5562-0cfc-11ea-abe1-c412f533dba1
1,606c5b29-0f88-11eb-ac1a-c412f533dba1,d12fe765-b444-11e9-abb3-c412f533dba1
2,1f389c6a-d5ab-11e9-abbe-c412f533dba1,43756fa4-57a0-11e9-ab9a-c412f533dba1
3,00000000-0000-0000-0000-000000000000,00000000-0000-0000-0000-000000000000
4,ac2c0012-6fd5-11e7-80fc-c412f533dba1,5dd88f32-6501-11ea-abf3-c412f533dba1
...,...,...
2291,ffa2180f-1f72-11eb-ac1d-c412f533dba1,b832aac3-fcda-11ea-ac18-c412f533dba1
2292,ffaa0816-16c4-11eb-ac1d-c412f533dba1,5dd88f32-6501-11ea-abf3-c412f533dba1
2293,377ea5e0-d089-11e9-abbd-c412f533dba1,ad52c7a8-a752-11e7-8115-c412f533dba1
2294,6dd39f15-d5a3-11e9-abbe-c412f533dba1,5dd88f32-6501-11ea-abf3-c412f533dba1


In [366]:
pandasql.sqldf("SELECT COUNT(l_client_id) FROM client_and_manager GROUP BY l_client_id HAVING COUNT(l_client_id)>1;", globals())

,COUNT(l_client_id)
0,17
1,2
2,2
3,2
4,3
...,...
83,2
84,2
85,2
86,2


We see that one client can have many managers. 

### Create spread sheet

In [367]:
# Find the first lead for each client
first_lead = leads[['l_client_id','created_at']].groupby(['l_client_id']).min()
first_lead.reset_index(inplace=True)

In [368]:
# Add indicator column of the first lead
leads = pd.merge(leads, first_lead, on='l_client_id', how='left')
leads['First'] = leads['created_at_x']==leads['created_at_y']
leads.drop(columns=['created_at_y'],inplace=True)
leads.rename(columns={"created_at_x": "created_at"},inplace=True)

In [369]:
# Find the first transaction for each client
first_trans = transactions[['l_client_id','created_at']].groupby(['l_client_id']).min()
first_trans.reset_index(inplace=True)

In [370]:
# Add indicator column of the first transaction
transactions = pd.merge(transactions, first_trans, on='l_client_id', how='left')
transactions['First'] = transactions['created_at_x']==transactions['created_at_y']
transactions.drop(columns=['created_at_y'],inplace=True)
transactions.rename(columns={"created_at_x": "created_at"},inplace=True)

Let's analyse the 0000 managers. Look at first lead, manager's record and purchase:

In [371]:
tmp = leads[(leads['l_manager_id']=='00000000-0000-0000-0000-000000000000') & (leads['First']==True)]['l_client_id'].to_list()
#tt = pandasql.sqldf("SELECT leads.l_client_id, leads.l_id, clients.l_manager_id, leads.created_at AS first_lead, clients.created_at AS manager_record FROM transactions, leads LEFT JOIN clients ON leads.l_client_id = clients.client_id WHERE leads.l_manager_id = '00000000-0000-0000-0000-000000000000';", globals())
#pandasql.sqldf("SELECT DISTINCT tt.*, transactions.created_at AS first_purchase FROM transactions INNER JOIN tt ON transactions.l_client_id = tt.l_client_id WHERE transactions.First = 1;", globals())

In [372]:
tmp1 = leads[leads['l_client_id'].isin(tmp)].copy()
tmp1.rename(columns={"l_client_id": "client_id"},inplace=True)
tmp1 =  pd.merge(tmp1, clients, on='client_id', how='left')
tmp1.rename(columns={"client_id": "l_client_id"},inplace=True)
tmp1

,lead_id,created_at_x,d_utm_medium,d_utm_source,l_manager_id_x,l_client_id,First,created_at_y,l_manager_id_y
0,028ef311-039d-11eb-ac18-c412f533dba1,2020-10-01 08:17:26,cpc,instagram,00000000-0000-0000-0000-000000000000,899b0ff5-0486-11eb-ac18-c412f533dba1,True,2020-10-02 12:09:05,1e9c5562-0cfc-11ea-abe1-c412f533dba1
1,03663558-0790-11eb-ac18-c412f533dba1,2020-10-06 08:54:28,social,vk,00000000-0000-0000-0000-000000000000,b031a66a-be52-11e7-8116-c412f533dba1,True,2017-10-31 00:00:00,db06b8bc-bfb2-11e7-8116-c412f533dba1
2,0ebe0d5b-0947-11eb-ac1a-c412f533dba1,2020-10-08 13:17:16,sms,viber,00000000-0000-0000-0000-000000000000,25bda8e3-d5ac-11e9-abbe-c412f533dba1,True,1900-01-01 00:00:00,9a87c608-52dc-11ea-abeb-c412f533dba1
3,110633fc-30ed-11eb-ac1f-c412f533dba1,2020-11-28 00:13:52,social,instagram,00000000-0000-0000-0000-000000000000,a0b393f9-9fb7-11e8-ab56-c412f533dba1,True,2018-08-14 00:00:00,43756fa4-57a0-11e9-ab9a-c412f533dba1
4,131d722d-1fec-11eb-ac1d-c412f533dba1,2020-11-06 08:53:56,cpc,instagram,b832aac3-fcda-11ea-ac18-c412f533dba1,660cf159-d323-11e9-abbe-c412f533dba1,False,2019-09-09 20:58:26,9a87c608-52dc-11ea-abeb-c412f533dba1
5,15e75ec4-078c-11eb-ac18-c412f533dba1,2020-10-06 08:26:21,cpc,instagram,00000000-0000-0000-0000-000000000000,88184830-6fd5-11e7-80fc-c412f533dba1,True,2016-11-12 00:00:00,db06b8bc-bfb2-11e7-8116-c412f533dba1
6,1b87b98e-078e-11eb-ac18-c412f533dba1,2020-10-06 08:40:50,utm_medium,unknown,00000000-0000-0000-0000-000000000000,1d9a669a-d5a5-11e9-abbe-c412f533dba1,True,1900-01-01 00:00:00,db06b8bc-bfb2-11e7-8116-c412f533dba1
7,2b77e10d-0c7b-11eb-ac1a-c412f533dba1,2020-10-12 15:07:52,,unknown,00000000-0000-0000-0000-000000000000,db8d4655-1379-11eb-ac1c-c412f533dba1,True,2020-10-21 12:46:07,9a87c608-52dc-11ea-abeb-c412f533dba1
8,2fd3ce3a-0946-11eb-ac1a-c412f533dba1,2020-10-08 13:11:02,sms,viber,00000000-0000-0000-0000-000000000000,2b0c485e-d5a6-11e9-abbe-c412f533dba1,True,1900-01-01 00:00:00,1bb0e7ca-5892-11ea-abeb-c412f533dba1
9,4189c9b6-0946-11eb-ac1a-c412f533dba1,2020-10-08 13:11:32,sms,viber,00000000-0000-0000-0000-000000000000,a49e0383-d5ad-11e9-abbe-c412f533dba1,True,1900-01-01 00:00:00,1bb0e7ca-5892-11ea-abeb-c412f533dba1


In [373]:
tmp1 = pd.merge(tmp1, transactions[(transactions['l_client_id'].isin(tmp)) & (transactions['First']==True)], on='l_client_id', how='left')

In [374]:
tmp1.sort_values(by=['l_client_id']).head(39)

,lead_id,created_at_x,d_utm_medium,d_utm_source,l_manager_id_x,l_client_id,First_x,created_at_y,l_manager_id_y,transaction_id,created_at,m_real_amount,First_y
24,a4ccc2e1-1da8-11eb-ac1d-c412f533dba1,2020-11-03 11:46:13,sms,viber,00000000-0000-0000-0000-000000000000,086ee5d4-620d-11ea-abf3-c412f533dba1,True,2020-03-09 17:50:43,abed5496-ca88-11e9-abb8-c412f533dba1,244185bc-de2f-4779-86be-102aea44e5da,2020-03-10 16:45:25,658.0,True
6,1b87b98e-078e-11eb-ac18-c412f533dba1,2020-10-06 08:40:50,utm_medium,unknown,00000000-0000-0000-0000-000000000000,1d9a669a-d5a5-11e9-abbe-c412f533dba1,True,1900-01-01 00:00:00,db06b8bc-bfb2-11e7-8116-c412f533dba1,NaN,NaT,NaN,NaN
27,cb228abb-1441-11eb-ac1d-c412f533dba1,2020-10-22 12:37:18,cpc,instagram,9a87c608-52dc-11ea-abeb-c412f533dba1,1da0523f-d5b5-11e9-abbe-c412f533dba1,False,1900-01-01 00:00:00,9a87c608-52dc-11ea-abeb-c412f533dba1,cdb98bbc-4bc7-4000-a731-4a5b763921ec,2020-10-23 13:34:36,31972.0,True
35,ed1c4705-1208-11eb-ac1c-c412f533dba1,2020-10-19 16:45:12,,unknown,9a87c608-52dc-11ea-abeb-c412f533dba1,1da0523f-d5b5-11e9-abbe-c412f533dba1,False,1900-01-01 00:00:00,9a87c608-52dc-11ea-abeb-c412f533dba1,cdb98bbc-4bc7-4000-a731-4a5b763921ec,2020-10-23 13:34:36,31972.0,True
29,d80fdd08-079c-11eb-ac18-c412f533dba1,2020-10-06 10:26:19,cpc,instagram,00000000-0000-0000-0000-000000000000,1da0523f-d5b5-11e9-abbe-c412f533dba1,True,1900-01-01 00:00:00,9a87c608-52dc-11ea-abeb-c412f533dba1,cdb98bbc-4bc7-4000-a731-4a5b763921ec,2020-10-23 13:34:36,31972.0,True
37,fa3a4501-272d-11eb-ac1d-c412f533dba1,2020-11-15 14:33:28,social,instagram,00000000-0000-0000-0000-000000000000,251d41aa-c96b-11e9-abb7-c412f533dba1,True,2019-08-28 00:00:00,bf340611-034b-11eb-ac18-c412f533dba1,NaN,NaT,NaN,NaN
2,0ebe0d5b-0947-11eb-ac1a-c412f533dba1,2020-10-08 13:17:16,sms,viber,00000000-0000-0000-0000-000000000000,25bda8e3-d5ac-11e9-abbe-c412f533dba1,True,1900-01-01 00:00:00,9a87c608-52dc-11ea-abeb-c412f533dba1,NaN,NaT,NaN,NaN
22,98031008-12c8-11eb-ac1c-c412f533dba1,2020-10-20 15:37:12,sms,viber,9a87c608-52dc-11ea-abeb-c412f533dba1,25bda8e3-d5ac-11e9-abbe-c412f533dba1,False,1900-01-01 00:00:00,9a87c608-52dc-11ea-abeb-c412f533dba1,NaN,NaT,NaN,NaN
8,2fd3ce3a-0946-11eb-ac1a-c412f533dba1,2020-10-08 13:11:02,sms,viber,00000000-0000-0000-0000-000000000000,2b0c485e-d5a6-11e9-abbe-c412f533dba1,True,1900-01-01 00:00:00,1bb0e7ca-5892-11ea-abeb-c412f533dba1,NaN,NaT,NaN,NaN
16,745baa8e-1920-11eb-ac1d-c412f533dba1,2020-10-28 17:21:15,,unknown,00000000-0000-0000-0000-000000000000,394eadaf-19dc-11eb-ac1d-c412f533dba1,True,2020-10-29 15:45:21,d6a49e22-f74f-11ea-ac18-c412f533dba1,NaN,NaT,NaN,NaN


1. From the first line, we see that transaction could happen without lead, but there was a manager.

2. We see that some transactions were created without a manager, but then the client was included in the database by a manager.

In [375]:
#leads = pandasql.sqldf("SELECT leads.lead_id, leads.created_at, leads.d_utm_medium, leads.d_utm_source, CASE WHEN leads.l_manager_id = '00000000-0000-0000-0000-000000000000' AND leads.l_client_id <> '00000000-0000-0000-0000-000000000000' THEN clients.l_manager_id ELSE leads.l_manager_id END AS l_manager_id, leads.l_client_id FROM leads LEFT JOIN clients ON leads.l_client_id=clients.client_id;", globals())

In [376]:
#leads['created_at'] = pd.to_datetime(leads['created_at'])

Select the specific time period 

In [377]:
# Select the data in the time period
result = leads[(leads['created_at']>period.x[0]) & (leads['created_at']<period.y[0])].copy()
#result = pandasql.sqldf("SELECT leads.created_at, leads.d_utm_source, CASE WHEN leads.l_manager_id = '00000000-0000-0000-0000-000000000000' THEN 'No manager' ELSE managers.d_manager END AS manager, CASE WHEN leads.l_manager_id = '00000000-0000-0000-0000-000000000000' THEN 'No club' ELSE managers.d_club END AS club, leads.l_client_id, CASE WHEN leads.l_client_id IN (SELECT DISTINCT leads.l_client_id FROM leads, period WHERE leads.created_at < period.x UNION SELECT DISTINCT transactions.l_client_id FROM transactions, period WHERE transactions.created_at < period.x) THEN 'Old' ELSE 'New' END AS New FROM period, leads LEFT JOIN managers ON leads.l_manager_id=managers.manager_id WHERE leads.created_at BETWEEN period.x AND period.y ORDER BY leads.l_client_id;", globals())

In [378]:
result.head()

,lead_id,created_at,d_utm_medium,d_utm_source,l_manager_id,l_client_id,First
4,009b1616-145d-11eb-ac1d-c412f533dba1,2020-10-22 15:52:04,cpc,vk,5dd88f32-6501-11ea-abf3-c412f533dba1,ac2c0012-6fd5-11e7-80fc-c412f533dba1,True
6,00aea163-13a5-11eb-ac1d-c412f533dba1,2020-10-21 17:54:59,,unknown,db06b8bc-bfb2-11e7-8116-c412f533dba1,00aea164-13a5-11eb-ac1d-c412f533dba1,True
7,00b288c8-1f2a-11eb-ac1d-c412f533dba1,2020-11-05 09:44:43,cpc,instagram,43756fa4-57a0-11e9-ab9a-c412f533dba1,9c1b0cfa-fd6b-11ea-ac18-c412f533dba1,True
8,00c4c14a-2456-11eb-ac1d-c412f533dba1,2020-11-11 23:42:17,cpc,google,5dd88f32-6501-11ea-abf3-c412f533dba1,bb2c46b7-24b5-11eb-ac1d-c412f533dba1,True
10,010e6800-1b56-11eb-ac1d-c412f533dba1,2020-10-31 12:49:37,cpc,instagram,db06b8bc-bfb2-11e7-8116-c412f533dba1,43836482-981e-11e9-abac-c412f533dba1,True


Date of the first purchase for each client:

In [379]:
result = pd.merge(result, transactions[transactions['First']==True], on='l_client_id', how='left')
result.drop(columns=['First_y', 'transaction_id'],inplace=True)
result['m_real_amount'] = result['m_real_amount'].fillna(0)
result['created_at_y'] = result['created_at_y'].fillna(datetime.datetime(2090, 1, 1))
result

,lead_id,created_at_x,d_utm_medium,d_utm_source,l_manager_id,l_client_id,First_x,created_at_y,m_real_amount
0,009b1616-145d-11eb-ac1d-c412f533dba1,2020-10-22 15:52:04,cpc,vk,5dd88f32-6501-11ea-abf3-c412f533dba1,ac2c0012-6fd5-11e7-80fc-c412f533dba1,True,2020-04-17 14:48:51,13040.0
1,00aea163-13a5-11eb-ac1d-c412f533dba1,2020-10-21 17:54:59,,unknown,db06b8bc-bfb2-11e7-8116-c412f533dba1,00aea164-13a5-11eb-ac1d-c412f533dba1,True,2090-01-01 00:00:00,0.0
2,00b288c8-1f2a-11eb-ac1d-c412f533dba1,2020-11-05 09:44:43,cpc,instagram,43756fa4-57a0-11e9-ab9a-c412f533dba1,9c1b0cfa-fd6b-11ea-ac18-c412f533dba1,True,2090-01-01 00:00:00,0.0
3,00c4c14a-2456-11eb-ac1d-c412f533dba1,2020-11-11 23:42:17,cpc,google,5dd88f32-6501-11ea-abf3-c412f533dba1,bb2c46b7-24b5-11eb-ac1d-c412f533dba1,True,2020-11-20 07:59:19,8349.0
4,010e6800-1b56-11eb-ac1d-c412f533dba1,2020-10-31 12:49:37,cpc,instagram,db06b8bc-bfb2-11e7-8116-c412f533dba1,43836482-981e-11e9-abac-c412f533dba1,True,2020-11-27 17:17:33,23641.0
...,...,...,...,...,...,...,...,...,...
1970,ff7d0045-12c7-11eb-ac1c-c412f533dba1,2020-10-20 15:32:56,sms,viber,9a87c608-52dc-11ea-abeb-c412f533dba1,2e4fbe67-12cb-11eb-ac1c-c412f533dba1,True,2090-01-01 00:00:00,0.0
1971,ffa2180e-1f72-11eb-ac1d-c412f533dba1,2020-11-05 18:27:15,,unknown,b832aac3-fcda-11ea-ac18-c412f533dba1,ffa2180f-1f72-11eb-ac1d-c412f533dba1,True,2020-11-05 15:29:21,747.0
1972,ffaa0815-16c4-11eb-ac1d-c412f533dba1,2020-10-25 17:21:33,,unknown,5dd88f32-6501-11ea-abf3-c412f533dba1,ffaa0816-16c4-11eb-ac1d-c412f533dba1,True,2090-01-01 00:00:00,0.0
1973,ffacb5af-194f-11eb-ac1d-c412f533dba1,2020-10-28 23:01:35,cpc,vk,ad52c7a8-a752-11e7-8115-c412f533dba1,377ea5e0-d089-11e9-abbd-c412f533dba1,True,2090-01-01 00:00:00,0.0


In [380]:
result.rename(columns={"created_at_y": "first_purch"},inplace=True)
result['no_purch_before'] = result['created_at_x'] < result['first_purch']

In [381]:
# Find clients with purchases
result['purchase'] = result.apply(lambda x : 'Yes' if (x['created_at_x']<transactions[transactions['l_client_id']==x['l_client_id']]['created_at']).any() and (transactions[transactions['l_client_id']==x['l_client_id']]['created_at']<x['created_at_x']+datetime.timedelta(days=7)).any() else 'No', axis=1)

In [382]:
result

,lead_id,created_at_x,d_utm_medium,d_utm_source,l_manager_id,l_client_id,First_x,first_purch,m_real_amount,no_purch_before,purchase
0,009b1616-145d-11eb-ac1d-c412f533dba1,2020-10-22 15:52:04,cpc,vk,5dd88f32-6501-11ea-abf3-c412f533dba1,ac2c0012-6fd5-11e7-80fc-c412f533dba1,True,2020-04-17 14:48:51,13040.0,False,No
1,00aea163-13a5-11eb-ac1d-c412f533dba1,2020-10-21 17:54:59,,unknown,db06b8bc-bfb2-11e7-8116-c412f533dba1,00aea164-13a5-11eb-ac1d-c412f533dba1,True,2090-01-01 00:00:00,0.0,True,No
2,00b288c8-1f2a-11eb-ac1d-c412f533dba1,2020-11-05 09:44:43,cpc,instagram,43756fa4-57a0-11e9-ab9a-c412f533dba1,9c1b0cfa-fd6b-11ea-ac18-c412f533dba1,True,2090-01-01 00:00:00,0.0,True,No
3,00c4c14a-2456-11eb-ac1d-c412f533dba1,2020-11-11 23:42:17,cpc,google,5dd88f32-6501-11ea-abf3-c412f533dba1,bb2c46b7-24b5-11eb-ac1d-c412f533dba1,True,2020-11-20 07:59:19,8349.0,True,No
4,010e6800-1b56-11eb-ac1d-c412f533dba1,2020-10-31 12:49:37,cpc,instagram,db06b8bc-bfb2-11e7-8116-c412f533dba1,43836482-981e-11e9-abac-c412f533dba1,True,2020-11-27 17:17:33,23641.0,True,No
...,...,...,...,...,...,...,...,...,...,...,...
1970,ff7d0045-12c7-11eb-ac1c-c412f533dba1,2020-10-20 15:32:56,sms,viber,9a87c608-52dc-11ea-abeb-c412f533dba1,2e4fbe67-12cb-11eb-ac1c-c412f533dba1,True,2090-01-01 00:00:00,0.0,True,No
1971,ffa2180e-1f72-11eb-ac1d-c412f533dba1,2020-11-05 18:27:15,,unknown,b832aac3-fcda-11ea-ac18-c412f533dba1,ffa2180f-1f72-11eb-ac1d-c412f533dba1,True,2020-11-05 15:29:21,747.0,False,No
1972,ffaa0815-16c4-11eb-ac1d-c412f533dba1,2020-10-25 17:21:33,,unknown,5dd88f32-6501-11ea-abf3-c412f533dba1,ffaa0816-16c4-11eb-ac1d-c412f533dba1,True,2090-01-01 00:00:00,0.0,True,No
1973,ffacb5af-194f-11eb-ac1d-c412f533dba1,2020-10-28 23:01:35,cpc,vk,ad52c7a8-a752-11e7-8115-c412f533dba1,377ea5e0-d089-11e9-abbd-c412f533dba1,True,2090-01-01 00:00:00,0.0,True,No


In [383]:
result['client_status'] = result.apply(lambda x : 'Trash' if x['l_client_id'] == '00000000-0000-0000-0000-000000000000' else x['First_x'], axis=1)

In [384]:
result['client_status'].replace([True, False], ['Yes', 'No'], inplace=True)
#result.drop(columns=['First_x'],inplace=True)
result['client_status'] = result.apply(lambda x : 'No' if (x['client_status'] == 'Yes') and (x['no_purch_before']==False) else x['client_status'], axis=1)

In [385]:
result.head(20)

,lead_id,created_at_x,d_utm_medium,d_utm_source,l_manager_id,l_client_id,First_x,first_purch,m_real_amount,no_purch_before,purchase,client_status
0,009b1616-145d-11eb-ac1d-c412f533dba1,2020-10-22 15:52:04,cpc,vk,5dd88f32-6501-11ea-abf3-c412f533dba1,ac2c0012-6fd5-11e7-80fc-c412f533dba1,True,2020-04-17 14:48:51,13040.0,False,No,No
1,00aea163-13a5-11eb-ac1d-c412f533dba1,2020-10-21 17:54:59,,unknown,db06b8bc-bfb2-11e7-8116-c412f533dba1,00aea164-13a5-11eb-ac1d-c412f533dba1,True,2090-01-01 00:00:00,0.0,True,No,Yes
2,00b288c8-1f2a-11eb-ac1d-c412f533dba1,2020-11-05 09:44:43,cpc,instagram,43756fa4-57a0-11e9-ab9a-c412f533dba1,9c1b0cfa-fd6b-11ea-ac18-c412f533dba1,True,2090-01-01 00:00:00,0.0,True,No,Yes
3,00c4c14a-2456-11eb-ac1d-c412f533dba1,2020-11-11 23:42:17,cpc,google,5dd88f32-6501-11ea-abf3-c412f533dba1,bb2c46b7-24b5-11eb-ac1d-c412f533dba1,True,2020-11-20 07:59:19,8349.0,True,No,Yes
4,010e6800-1b56-11eb-ac1d-c412f533dba1,2020-10-31 12:49:37,cpc,instagram,db06b8bc-bfb2-11e7-8116-c412f533dba1,43836482-981e-11e9-abac-c412f533dba1,True,2020-11-27 17:17:33,23641.0,True,No,Yes
5,012af3e3-1faa-11eb-ac1d-c412f533dba1,2020-11-06 01:00:59,cpc,instagram,db06b8bc-bfb2-11e7-8116-c412f533dba1,cf3762bf-f5f9-11e8-ab84-c412f533dba1,False,2020-03-03 17:41:22,262.0,False,Yes,No
6,0165bb18-2743-11eb-ac1d-c412f533dba1,2020-11-15 17:03:56,cpc,instagram,db06b8bc-bfb2-11e7-8116-c412f533dba1,f683a91a-59d5-11e9-ab9c-c412f533dba1,True,2090-01-01 00:00:00,0.0,True,No,Yes
7,016f5641-21c0-11eb-ac1d-c412f533dba1,2020-11-08 16:43:31,,unknown,b832aac3-fcda-11ea-ac18-c412f533dba1,2502c57d-21ae-11eb-ac1d-c412f533dba1,False,2020-11-08 13:47:18,197.0,False,Yes,No
8,01769e26-19dd-11eb-ac1d-c412f533dba1,2020-10-29 15:51:05,cpc,instagram,ad52c7a8-a752-11e7-8115-c412f533dba1,00000000-0000-0000-0000-000000000000,False,2090-01-01 00:00:00,0.0,True,No,Trash
9,018378bf-1796-11eb-ac1d-c412f533dba1,2020-10-26 18:17:41,,unknown,43756fa4-57a0-11e9-ab9a-c412f533dba1,1d7e0076-1792-11eb-ac1d-c412f533dba1,False,2020-10-29 17:41:18,40739.0,True,Yes,No


In [386]:
result['m_real_amount'] = result.apply(lambda x : 0 if x['no_purch_before'] == False else x['m_real_amount'], axis=1)
result

,lead_id,created_at_x,d_utm_medium,d_utm_source,l_manager_id,l_client_id,First_x,first_purch,m_real_amount,no_purch_before,purchase,client_status
0,009b1616-145d-11eb-ac1d-c412f533dba1,2020-10-22 15:52:04,cpc,vk,5dd88f32-6501-11ea-abf3-c412f533dba1,ac2c0012-6fd5-11e7-80fc-c412f533dba1,True,2020-04-17 14:48:51,0.0,False,No,No
1,00aea163-13a5-11eb-ac1d-c412f533dba1,2020-10-21 17:54:59,,unknown,db06b8bc-bfb2-11e7-8116-c412f533dba1,00aea164-13a5-11eb-ac1d-c412f533dba1,True,2090-01-01 00:00:00,0.0,True,No,Yes
2,00b288c8-1f2a-11eb-ac1d-c412f533dba1,2020-11-05 09:44:43,cpc,instagram,43756fa4-57a0-11e9-ab9a-c412f533dba1,9c1b0cfa-fd6b-11ea-ac18-c412f533dba1,True,2090-01-01 00:00:00,0.0,True,No,Yes
3,00c4c14a-2456-11eb-ac1d-c412f533dba1,2020-11-11 23:42:17,cpc,google,5dd88f32-6501-11ea-abf3-c412f533dba1,bb2c46b7-24b5-11eb-ac1d-c412f533dba1,True,2020-11-20 07:59:19,8349.0,True,No,Yes
4,010e6800-1b56-11eb-ac1d-c412f533dba1,2020-10-31 12:49:37,cpc,instagram,db06b8bc-bfb2-11e7-8116-c412f533dba1,43836482-981e-11e9-abac-c412f533dba1,True,2020-11-27 17:17:33,23641.0,True,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...
1970,ff7d0045-12c7-11eb-ac1c-c412f533dba1,2020-10-20 15:32:56,sms,viber,9a87c608-52dc-11ea-abeb-c412f533dba1,2e4fbe67-12cb-11eb-ac1c-c412f533dba1,True,2090-01-01 00:00:00,0.0,True,No,Yes
1971,ffa2180e-1f72-11eb-ac1d-c412f533dba1,2020-11-05 18:27:15,,unknown,b832aac3-fcda-11ea-ac18-c412f533dba1,ffa2180f-1f72-11eb-ac1d-c412f533dba1,True,2020-11-05 15:29:21,0.0,False,No,No
1972,ffaa0815-16c4-11eb-ac1d-c412f533dba1,2020-10-25 17:21:33,,unknown,5dd88f32-6501-11ea-abf3-c412f533dba1,ffaa0816-16c4-11eb-ac1d-c412f533dba1,True,2090-01-01 00:00:00,0.0,True,No,Yes
1973,ffacb5af-194f-11eb-ac1d-c412f533dba1,2020-10-28 23:01:35,cpc,vk,ad52c7a8-a752-11e7-8115-c412f533dba1,377ea5e0-d089-11e9-abbd-c412f533dba1,True,2090-01-01 00:00:00,0.0,True,No,Yes


In [387]:
result.rename(columns={"l_manager_id": "manager_id"},inplace=True)
df_inner_1 = pd.merge(result, managers, on='manager_id', how='left')
#df_inner_1 = pandasql.sqldf("SELECT DISTINCT result.created_at_x, result.l_manager_id, result.d_utm_source, CASE WHEN result.l_manager_id = '00000000-0000-0000-0000-000000000000' THEN 'No manager' ELSE managers.d_manager END AS manager, CASE WHEN result.l_manager_id = '00000000-0000-0000-0000-000000000000' THEN 'No club' ELSE managers.d_club END AS club, result.client_status, result.no_purch_before FROM result LEFT JOIN managers ON result.l_manager_id=managers.manager_id;", globals())

In [388]:
df_inner_1['d_manager'] = df_inner_1['d_manager'].fillna('unknown')
df_inner_1['d_club'] = df_inner_1['d_club'].fillna('unknown')
df_inner_1['m_real_amount'] = df_inner_1['m_real_amount'].astype(int)
df_inner_1

,lead_id,created_at_x,d_utm_medium,d_utm_source,manager_id,l_client_id,First_x,first_purch,m_real_amount,no_purch_before,purchase,client_status,d_manager,d_club
0,009b1616-145d-11eb-ac1d-c412f533dba1,2020-10-22 15:52:04,cpc,vk,5dd88f32-6501-11ea-abf3-c412f533dba1,ac2c0012-6fd5-11e7-80fc-c412f533dba1,True,2020-04-17 14:48:51,0,False,No,No,manager #10,club #2
1,00aea163-13a5-11eb-ac1d-c412f533dba1,2020-10-21 17:54:59,,unknown,db06b8bc-bfb2-11e7-8116-c412f533dba1,00aea164-13a5-11eb-ac1d-c412f533dba1,True,2090-01-01 00:00:00,0,True,No,Yes,manager #4,club #2
2,00b288c8-1f2a-11eb-ac1d-c412f533dba1,2020-11-05 09:44:43,cpc,instagram,43756fa4-57a0-11e9-ab9a-c412f533dba1,9c1b0cfa-fd6b-11ea-ac18-c412f533dba1,True,2090-01-01 00:00:00,0,True,No,Yes,manager #11,club #4
3,00c4c14a-2456-11eb-ac1d-c412f533dba1,2020-11-11 23:42:17,cpc,google,5dd88f32-6501-11ea-abf3-c412f533dba1,bb2c46b7-24b5-11eb-ac1d-c412f533dba1,True,2020-11-20 07:59:19,8349,True,No,Yes,manager #10,club #2
4,010e6800-1b56-11eb-ac1d-c412f533dba1,2020-10-31 12:49:37,cpc,instagram,db06b8bc-bfb2-11e7-8116-c412f533dba1,43836482-981e-11e9-abac-c412f533dba1,True,2020-11-27 17:17:33,23641,True,No,Yes,manager #4,club #2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970,ff7d0045-12c7-11eb-ac1c-c412f533dba1,2020-10-20 15:32:56,sms,viber,9a87c608-52dc-11ea-abeb-c412f533dba1,2e4fbe67-12cb-11eb-ac1c-c412f533dba1,True,2090-01-01 00:00:00,0,True,No,Yes,manager #9,club #3
1971,ffa2180e-1f72-11eb-ac1d-c412f533dba1,2020-11-05 18:27:15,,unknown,b832aac3-fcda-11ea-ac18-c412f533dba1,ffa2180f-1f72-11eb-ac1d-c412f533dba1,True,2020-11-05 15:29:21,0,False,No,No,manager #13,club #3
1972,ffaa0815-16c4-11eb-ac1d-c412f533dba1,2020-10-25 17:21:33,,unknown,5dd88f32-6501-11ea-abf3-c412f533dba1,ffaa0816-16c4-11eb-ac1d-c412f533dba1,True,2090-01-01 00:00:00,0,True,No,Yes,manager #10,club #2
1973,ffacb5af-194f-11eb-ac1d-c412f533dba1,2020-10-28 23:01:35,cpc,vk,ad52c7a8-a752-11e7-8115-c412f533dba1,377ea5e0-d089-11e9-abbd-c412f533dba1,True,2090-01-01 00:00:00,0,True,No,Yes,manager #7,club #1


In [389]:
df_inner_1 = df_inner_1[['d_utm_source','d_club', 'd_manager', 'client_status', 'purchase', 'm_real_amount']].copy()
df_inner_1

,d_utm_source,d_club,d_manager,client_status,purchase,m_real_amount
0,vk,club #2,manager #10,No,No,0
1,unknown,club #2,manager #4,Yes,No,0
2,instagram,club #4,manager #11,Yes,No,0
3,google,club #2,manager #10,Yes,No,8349
4,instagram,club #2,manager #4,Yes,No,23641
...,...,...,...,...,...,...
1970,viber,club #3,manager #9,Yes,No,0
1971,unknown,club #3,manager #13,No,No,0
1972,unknown,club #2,manager #10,Yes,No,0
1973,vk,club #1,manager #7,Yes,No,0


In [390]:
df_inner_1['Trash'] = df_inner_1['client_status'] == 'Trash'
df_inner_1['New_customer'] = df_inner_1['client_status'] == 'Yes'

In [391]:
df_inner_1['new_purchase'] = result.apply(lambda x : 1 if (x['no_purch_before'] == True) and (x['purchase']=='Yes') else 0, axis=1)
df_inner_1

,d_utm_source,d_club,d_manager,client_status,purchase,m_real_amount,Trash,New_customer,new_purchase
0,vk,club #2,manager #10,No,No,0,False,False,0
1,unknown,club #2,manager #4,Yes,No,0,False,True,0
2,instagram,club #4,manager #11,Yes,No,0,False,True,0
3,google,club #2,manager #10,Yes,No,8349,False,True,0
4,instagram,club #2,manager #4,Yes,No,23641,False,True,0
...,...,...,...,...,...,...,...,...,...
1970,viber,club #3,manager #9,Yes,No,0,False,True,0
1971,unknown,club #3,manager #13,No,No,0,False,False,0
1972,unknown,club #2,manager #10,Yes,No,0,False,True,0
1973,vk,club #1,manager #7,Yes,No,0,False,True,0


In [392]:
df_inner_1.replace([True, False], [1, 0], inplace=True)

In [394]:
import pygsheets

In [395]:
gc = pygsheets.authorize(service_file='write_key.json')

In [396]:
sh = gc.open('Report Anton Selitskiy')

In [397]:
wks = sh[0]

In [398]:
wks.set_dataframe(df_inner_1,(1,1),copy_head=True)